### ETL para el dataset de la funcion UserForGenre

In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [3]:
steam = pd.read_json('../Dataset_clean/Steam_Games_Limpio.json.gz', compression='gzip')
review = pd.read_json('../Dataset_clean/User_Reviews_Limpio.json.gz', compression='gzip')
items = pq.read_table('Datasets OPS/items.parquet').to_pandas()

In [4]:
#Nos quedamos la columna que utilizaremos
generosAVer = steam['genres']

In [5]:
# Guardamos en la lista de genelos Todos los generos que existen en el dataset
listaDeGeneros = []
for linea in generosAVer:
  for i in linea:
    if i not in listaDeGeneros:
      listaDeGeneros.append(i)

generos = {'genero':listaDeGeneros}
df = pd.DataFrame(generos)
df[['usuario','Horas jugadas']] = None

In [6]:
user_items = items 
itemsDeUsuario = items.explode('items')
userID = itemsDeUsuario['user_id']
itemsDeUsuario = pd.json_normalize(itemsDeUsuario['items'])
itemsDeUsuario['user_id'] = list(userID)
itemsDeUsuario.drop(columns=['playtime_2weeks','item_name'], inplace=True)
itemsDeUsuario.dropna(subset='item_id', inplace=True)

In [7]:
# Unimos los items de todos los usuarios con los que tenemos informacion en el dataset de steamgames
itemsDeUsuario['item_id'] = itemsDeUsuario['item_id'].astype(int)
itemsDeUsuario = itemsDeUsuario.merge(steam[['id','genres','release_date']], left_on='item_id', right_on='id')
itemsDeUsuario

,item_id,playtime_forever,user_id,id,genres,release_date
0,10,6.0,76561197970982479,10,[Action],2000
1,20,0.0,76561197970982479,20,[Action],1999
2,30,7.0,76561197970982479,30,[Action],2003
3,40,0.0,76561197970982479,40,[Action],2001
4,50,0.0,76561197970982479,50,[Action],1999
...,...,...,...,...,...,...
4186243,304930,677.0,76561198329548331,304930,"[Action, Adventure, Casual, Free to Play, Indie]",2017
4186244,227940,43.0,76561198329548331,227940,"[Action, Free to Play, Indie, Massively Multip...",2016
4186245,346330,0.0,76561198329548331,346330,"[Action, Adventure, Free to Play, Indie, Early...",2016
4186246,388490,3.0,76561198329548331,388490,"[Adventure, Free to Play]",2015


In [8]:
#Calcular por genero cual es el usuario que mas jugo a tal genero, por horas por año de lanzamiento
for i in df['genero'].values:
    genero = i

    #Juegos del genero pedido
    mascara = itemsDeUsuario['genres'].apply(lambda x: genero in x)

    #Calcular las horas que tiene un usuario con juegos del genero
    sumaDehoras = itemsDeUsuario[mascara].groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False)
    
    if not sumaDehoras.empty:
        usuario = sumaDehoras.head(1).index[0]
    else:
        usuario = None

    df.loc[df['genero'] == genero, 'usuario'] = usuario

    aniosYhoras = itemsDeUsuario[(itemsDeUsuario['user_id'] == usuario) & mascara].groupby('release_date')['playtime_forever'].sum()
    anio = aniosYhoras.index
    horasJugadas = {}
    for horas, anio in enumerate(anio):
        horasJugadas[f'Año {anio}'] = f'Horas: {aniosYhoras.iloc[horas]}'

    df.loc[df['genero'] == genero, 'Horas jugadas'] = [horasJugadas]
    
    
    # Recuerda borrar/ depurar la celda para que no pese mucho el notebook y se suba al repo, al finalizar y exportar el dataset

In [9]:
df.dropna(subset=usuario, inplace=True)
df.reset_index(drop=True,inplace=True)

In [10]:
df['genero'] = df['genero'].apply(lambda x: x.capitalize())

In [12]:
df.to_json("../Dataset_clean/genre.json.gz", compression='gzip')
#exportamos